In [ ]:
#pip install langchain

In [ ]:
#pip install sentence-transformers

In [ ]:
#pip install datasets


In [ ]:
#pip install faiss-gpu

In [ ]:
import warnings
warnings.filterwarnings('ignore')

We will use **orca-math-word-problems-200k**, dataset contains ~200K grade school math word problems. All the answers in this dataset is generated using Azure GPT4-Turbo.

[Source](https://huggingface.co/datasets/microsoft/orca-math-word-problems-200k)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("microsoft/orca-math-word-problems-200k")

Generating train split:   0%|          | 0/200035 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

After loading the data we are converting it to pandas dataframe for further use.

In [ ]:
pandas_data = pd.DataFrame(dataset['train'])

In [ ]:
pandas_data.head(2)


,question,answer
0,Jungkook is the 5th place. Find the number of ...,"If Jungkook is in 5th place, then 4 people cro..."
1,A number divided by 10 is 6. Yoongi got the re...,"Let's call the certain number ""x"". According t..."


In [ ]:
dataforIndexing=pandas_data.iloc[0:4]

Here we are taking only 4 rows to be processed, because for whole data **embedding** can take much time based on your system configuration[RAM]

In [ ]:
dataforIndexing.head(4)

,question,answer
0,Jungkook is the 5th place. Find the number of ...,"If Jungkook is in 5th place, then 4 people cro..."
1,A number divided by 10 is 6. Yoongi got the re...,"Let's call the certain number ""x"". According t..."
2,Dongju selects a piece of paper with a number ...,To find the second smallest and third smallest...
3,"You wanted to subtract 46 from a number, but y...",If you accidentally subtracted 59 instead of 4...


This is another way of loading the data by passing the col to the **HuggingFaceDatasetLoader**, here we will have less control compare to above approach

In [ ]:
dataset_name='microsoft/orca-math-word-problems-200k'
dataset_columns='answer'

In [ ]:
from langchain.document_loaders import HuggingFaceDatasetLoader

In [ ]:
# Create a loader instance
#loader = HuggingFaceDatasetLoader(dataset_name, dataset_columns)
loader = HuggingFaceDatasetLoader(dataset_name,dataset_columns) #

In [ ]:
data=loader.load()

**End** of HuggingFaceDatasetLoader

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
txt=''
for ans in dataforIndexing.answer:
  txt+=ans

In [ ]:
type(txt)

str

In [ ]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
#docs = text_splitter.split_documents(data)
docs=text_splitter.split_text(txt)

In [ ]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'} # you can switch the device based on your system

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs,
    # Pass the encoding options
)

In [ ]:
db = FAISS.from_texts(docs, embeddings)
db.save_local(folder_path="/content/faiss_db", index_name="myFaissIndex")

In [ ]:
#Once vector store and Faiss index created, you can load that index and search your query

db = FAISS.load_local(folder_path="/content/faiss_db",embeddings=embeddings,index_name="myFaissIndex",allow_dangerous_deserialization=True)
searchDocs = db.similarity_search("You wanted to subtract 46 from a number, but you accidentally subtract 59 and get 43. How much do you get from the correct calculation?")
print(searchDocs[0].page_content)

The possible three-digit numbers are:
- 168
- 186
- 618
- 681
- 816
- 861

Now, let's arrange these numbers in ascending order to find the second smallest and third smallest numbers:
- 168 (smallest)
- 186 (second smallest)
- 618 (third smallest)
- 681
- 816
- 861 (largest)

The second smallest number is 186, and the third smallest number is 618.

Now, we find the sum of the second smallest and third smallest numbers:
186 + 618 = 804

Therefore, the sum of the second smallest and third smallest three-digit numbers that Dongju can make with the digits 1, 6, and 8 is 804.If you accidentally subtracted 59 instead of 46 and got 43, you can find the original number by adding 59 back to 43:

43 + 59 = 102

Now, to find the result of the correct calculation, subtract 46 from the original number:

102 - 46 = 56

So, if you subtract 46 from the original number, you would get 56.
